In [1]:
from functions import get_points, write_to_laz #, recursive_plane

import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import pdal
import shapely.wkt
from numpy.lib import recfunctions as rfn

import numpy as np
import pandas as pd
import matplotlib
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pyntcloud import PyntCloud
from sklearn.cluster import DBSCAN

from scipy.spatial import cKDTree

from pyntcloud.ransac.fitters import single_fit
from pyntcloud.ransac.models import RansacPlane

In [2]:
READ_PIPELINE = """
{{
    "pipeline": [
        {{
            "type": "readers.ept",
            "filename": "{path}",
            "bounds": "{bounds}"
        }},
        {{
            "type":"filters.crop",
            "polygon":"{wkt}"
        }}
    ]
}}
"""

WRITE_PIPELINE = """
{{
    "pipeline": [
        {{
            "type": "writers.las",
            "filename": "{path}",
            "extra_dims": "all"
        }}
    ]
}}
"""


def get_points(ept_path, bounds, wkt):
    pipeline = pdal.Pipeline(READ_PIPELINE.format(
        path=ept_path,
        bounds=bounds,
        wkt=wkt)
    )

    pipeline.validate()
    pipeline.execute()
    point_cloud = pipeline.arrays[0]

    return point_cloud


def write_to_laz(point_cloud, path):
    pipeline = pdal.Pipeline(
        WRITE_PIPELINE.format(path=path),
        arrays=[point_cloud]
    )
    pipeline.validate()
    pipeline.execute()


def hausdorff_distance(reference_cloud, compared_cloud):
    """

    Parameters
    ----------
    reference_cloud : (Mx3) array
        The X, Y, Z coordinates of points in the reference cloud
    compared_cloud : (Mx3) array
        The X, Y, Z coordinates of points in the compared cloud
    """
    tree = cKDTree(reference_cloud)
    distances, _ = tree.query(compared_cloud, k=1)
    return distances

def filter_distance(reference_cloud, compared_cloud, max_dist = 0.05):
    reference_array = np.array(reference_cloud[['X', 'Y', 'Z']].tolist())
    compared_array = np.array(compared_cloud[['X', 'Y', 'Z']].tolist())

    haus_distances = hausdorff_distance(reference_array, 
                                        compared_array)
    out_array = rfn.append_fields(compared_cloud[['X','Y','Z','Red','Green','Blue']], 
                                'Distance', 
                                haus_distances)
    filtered_out_array = out_array[out_array['Distance'] < max_dist]

    return filtered_out_array





In [7]:
# test_area = [117813.6,487266.1,117942.1,487370.7]
# xmin,ymin,xmax,ymax = test_area

path_a10_2018 = '/var/data/rws/data/2018/entwined/ept.json'
path_a10_2019 = '/var/data/rws/data/2019/amsterdam_entwined/ept.json'

# random area: 
# wkt = 'POLYGON((117813.6 487266.1,117813.6 487370.7,117942.1 487370.7,117942.1 487266.1,117813.6 487266.1))'

# small area: 
# wkt = 'POLYGON((117813 487266,117813 487270,117815 487270,117815 487266,117813 487266))' # small! 

# portal:
# wkt = 'POLYGON((118032.6 488782.8,118032.6 488793.5,118078.8 488793.5,118078.8 488782.8,118032.6 488782.8))' # portal

# portal zonder palen
# wkt = 'POLYGON((118037.0 488782.8,118037.0 488793.5,118072.0 488793.5,118072.0 488782.8,118037.0 488782.8))' # portal zonder palen

# brug
wkt = 'POLYGON((117875.3 487304,117875.3 487329,117906.1 487329,117906.1 487304,117875.3 487304))' # brug

# stukje brug
# wkt = 'POLYGON((117890.1 487311,117890.1 487312.5,117892 487312.5,117892 487311,117890.1 487311))' # brug


xmin,ymin,xmax,ymax = shapely.wkt.loads(wkt).bounds

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc18 = get_points(path_a10_2018, bounds, wkt)
print(f'loaded pc 2018')

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc19 = get_points(path_a10_2019, bounds, wkt)
print(f'loaded pc 2019')

loaded pc 2018
loaded pc 2019


In [8]:
filtered_2018 = filter_distance(pc18, pc19)
print('filtered')
points_2018 = pd.DataFrame(
    np.array(filtered_2018[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])
print('dataframed')

filtered
dataframed


In [ ]:
try:
    del points_with_planes
except:
    pass
tst = recursive_planes(points_2018.copy(), 2, 10000)
print('fit planes')

ransac = rfn.append_fields(filtered_2018[['X','Y','Z','Red','Green','Blue']], 
                              'Classification', 
                              tst['cid'].astype(np.uint16),
                              usemask=False
                             )

write_to_laz(ransac, '/var/data/rws/data/output/tst_ransac/brug_planes.laz')
print('wrote')



In [33]:
np.unique(ransac['Classification'], return_counts=True)

(array([ 1,  2, 10], dtype=uint16), array([330368, 198809, 466872]))

In [16]:
xyz = PyntCloud(pd.DataFrame({
                'x':points_2018.x,
                'y':points_2018.y,
                'z':points_2018.z
                   }))

inliers, best_model = single_fit(xyz.points.values, 
                                 RansacPlane, 
                                 return_model=True,
                                 #max_dist=0.2, 
                                 max_iterations=1000, 
                                 n_inliers_to_stop=None)
            
print(best_model.point)
print(best_model.normal)

[ 1.17893703e+05  4.87315456e+05 -1.50178186e-01]
[ 0.03275589 -0.0117713   0.99939406]


In [30]:
outliers = best_model.get_projections(xyz.points.values)[0] <= 0.2
points_2018[outliers].shape

(333161, 6)

In [34]:

def recursive_planes(pyntcloud_pts, n_planes = 2, min_pts = 100, max_dist = 0.2, max_iterations = 10000):
    
    pyntcloud_pts['uid'] = pyntcloud_pts.index
    ransac_points = pyntcloud_pts.copy()
    points_with_planes = None
    
    for i in range(n_planes):
        if len(pyntcloud_pts.index) < min_pts:
            print(f'found {i + 1} planes')
            break
            
        else:
            # fit a plane
            # ransac_points.add_scalar_field("plane_fit",
            #                 max_dist=0.2, 
            #                max_iterations=1000, 
            #                n_inliers_to_stop=None)
            
            xyz = PyntCloud(pd.DataFrame({
                'x':ransac_points.x,
                'y':ransac_points.y,
                'z':ransac_points.z
                   }))
            
            inliers, best_model = single_fit(xyz.points.values, 
                                             RansacPlane, 
                                             return_model=True,
                                             max_iterations=max_iterations, 
                                             n_inliers_to_stop=None)
            best_inliers = best_model.get_projections(xyz.points.values)[0] < max_dist
            
            print(best_model.point)
            print(best_model.normal)
            

                ## TO DOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
                ## the inliers are not correctly classified, 
                ## probably because max dist is not a argument for single_fit. 
                ## figure this out! 
                # distances to plane = best_model.get_projections(xyz.points.values)

            # create frame of uid and plane
            ransacplane = pd.DataFrame({
                'uid':ransac_points.uid,  
                'plane': best_inliers.astype(np.int)
            })
            
            # Copy the non-planes
            outliers = best_model.get_projections(xyz.points.values)[0] >= max_dist
            ransacrest = ransac_points[outliers]
            print(np.unique(outliers, return_counts=True))
            
            # if oints_with_planes exists, merge the existing planes with the new found plane
            if points_with_planes is not None:
                points_with_planes = pd.merge(  points_with_planes, 
                                                ransacplane, 
                                                on='uid',
                                                how='left')
            
            # If it does not exists, merge the initial pointcloud with found ransacplane
            else:
                points_with_planes = pd.merge(  pyntcloud_pts, 
                                                ransacplane, 
                                                on='uid',
                                                how='left')
            # if there is no column with cid, create a new one and give it all the value 10
            if i == 0:
                points_with_planes['cid'] = 10
            
            # at the 
            # points_with_planes['cid'] = points_with_planes.plane.where(points_with_planes['plane'] == 1, i+1)
            points_with_planes['cid'] = np.where(points_with_planes['plane'] == 1, i+1, points_with_planes['cid'])
            points_with_planes = points_with_planes.drop(['plane'], axis=1)

            ransac_points = ransacrest.copy()
            
    return points_with_planes


